In [1]:
import os
from IPython.display import clear_output
os.chdir('..')

In [2]:
os.environ["CONFIG"] = "configs/small/small_p_fold.yaml"

In [3]:
%%bash
cd data
chmod +x get_dataset.sh
./get_dataset.sh

mkdir: cannot create directory ‘raw’: File exists


In [4]:
!python3 data/prepare_data.py --config_path $CONFIG \
    --sec_req data/raw/SecReq --promise data/raw/nfr/nfr.arff \
    --concord data/raw/NFRClassifier/gate/application-resources/Requirements/ \
    --cchit data/raw/CCHIT.xls --owasp data/raw/OWASP -o data/processed

In [5]:
from constants import DATASETS_CONFIG_PATH

pfold = [[3, 9, 11], [1, 5, 12], [6, 10, 13], [1, 8, 14], [3, 12, 15], [2, 5, 11], [6, 9, 14], [7, 8, 13], [2, 4, 15], [4, 7, 10]]

all_files = list(range(1, 16))

for test_files in pfold:
  train_files = [str(j) for j in all_files if j not in test_files]
  test_files = list(map(str, test_files))
  print(train_files, test_files)
  train_files_config = "\n  ".join([
    f"    - {file_}" for file_ in train_files
  ])
  test_files_config = "\n  ".join([
    f"    - {file_}" for file_ in test_files
  ])
  configuration = f"""
  train_datasets:
    promise:
  {train_files_config}
  valid_datasets:
    promise:
  {test_files_config}
"""

  with open(DATASETS_CONFIG_PATH, "w+") as f:
    f.write(configuration)

  !CUBLAS_WORKSPACE_CONFIG=:4096:8 python3 train.py $CONFIG


['1', '2', '4', '5', '6', '7', '8', '10', '12', '13', '14', '15'] ['3', '9', '11']
  0%|                                                  | 0/1580 [00:00<?, ?it/s]Traceback (most recent call last):
  File "train.py", line 181, in <module>
    train_and_evaluate(model_type=configuration.model_type,
  File "train.py", line 145, in train_and_evaluate
    metrics = train(model_type, epochs, labels_data, seed)
  File "train.py", line 75, in train
    trainer.train()
  File "/home/hawk/anaconda3/envs/thesis/lib/python3.8/site-packages/transformers/trainer.py", line 975, in train
    self.optimizer.step()
  File "/home/hawk/anaconda3/envs/thesis/lib/python3.8/site-packages/torch/optim/lr_scheduler.py", line 65, in wrapper
    return wrapped(*args, **kwargs)
  File "/home/hawk/anaconda3/envs/thesis/lib/python3.8/site-packages/torch/optim/optimizer.py", line 89, in wrapper
    return func(*args, **kwargs)
  File "/home/hawk/anaconda3/envs/thesis/lib/python3.8/site-packages/transformers/optimiza